開始訓練ReID

In [1]:
#!/usr/bin/env python
# encoding: utf-8
"""
@author:  sherlock
@contact: sherlockliao01@gmail.com
"""
import sys
import os

# 設定工作目錄
working_directory = r'E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main'
os.chdir(working_directory)
print("Current Working Directory:", os.getcwd())

sys.path.append('.')

from fast_reid.fastreid.config import get_cfg
from fast_reid.fastreid.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from fast_reid.fastreid.utils.checkpoint import Checkpointer

def setup(args):
    """
    創建配置並進行基本設置。
    """
    cfg = get_cfg()
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

def main(args):

    cfg = setup(args)

    if args.eval_only:
        cfg.defrost()
        cfg.MODEL.BACKBONE.PRETRAIN = False
        model = DefaultTrainer.build_model(cfg)
        print(model)
        Checkpointer(model).load(cfg.MODEL.WEIGHTS)  # 載入已訓練模型

        res = DefaultTrainer.test(cfg, model)
        return res

    trainer = DefaultTrainer(cfg)

    trainer.resume_or_load(resume=args.resume)

    return trainer.train()

class Args:
    config_file = 'fast_reid/configs/AICUP/bagtricks_R50-ibn.yml'
    eval_only = False
    resume = False
    opts = []
    num_gpus = 1
    num_machines = 1
    machine_rank = 0
    dist_url = "tcp://127.0.0.1:12345"

args = Args()

print("Command Line Args:", args)

launch(
    main,
    args.num_gpus,
    num_machines=args.num_machines,
    machine_rank=args.machine_rank,
    dist_url=args.dist_url,
    args=(args,),
)
